In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights
import torch

In [5]:
mGPT_model = AutoModelForCausalLM.from_pretrained('ai-forever/mGPT-13B', device_map="auto", offload_folder="offload", torch_dtype=torch.float16)

Loading checkpoint shards:  83%|████████▎ | 5/6 [00:41<00:10, 10.29s/it]

: 

In [4]:
with init_empty_weights():
    mGPT_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT-13B", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]/home/norrman/anaconda3/envs/thesis/lib/python3.9/site-packages/torch/nn/modules/module.py:2025: UserWarning: for transformer.wte.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/norrman/anaconda3/envs/thesis/lib/python3.9/site-packages/torch/nn/modules/module.py:2025: UserWarning: for transformer.wpe.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying fr

In [ ]:
# mGPT_pipe = pipeline("text-generation", model="ai-forever/mGPT-13B")


# mGPT_tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT-13B")
# mGPT_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT-13B")

In [18]:
XGLM_tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
XGLM_model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")

XGLM_pipe = pipeline("text-generation", model="facebook/xglm-564M")

In [12]:
context = 'Emma and Monica are not doctors, but Mark and Andrew are.'
question = 'Who are doctors?'
answer = 'Mark and Andrew.'

inputs = XGLM_tokenizer(context + ' ' + question, return_tensors='pt')
inputs

{'input_ids': tensor([[     2,  37248,     53,  87674,    256,    422, 128590,      4,    540,
           7102,     53,  36890,    256,      5,  16850,    256, 128590,     28]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
outputs = XGLM_model(**inputs, labels=inputs['input_ids'])
outputs 

CausalLMOutputWithCrossAttentions(loss=tensor(8.0554, grad_fn=<NllLossBackward0>), logits=tensor([[[-2.6019, -2.7024, 23.7592,  ..., -2.3627, -2.8033, -2.4592],
         [-1.4118, -2.1522, 33.1611,  ..., -0.9947, -2.3361, -2.4036],
         [-1.2028, -2.0319, 37.6757,  ..., -1.2416, -2.1702, -2.4323],
         ...,
         [-0.4248, -1.3479, 58.3325,  ...,  0.3015, -1.3919, -2.2220],
         [ 0.2053, -0.8029, 59.3268,  ...,  0.8111, -0.8421, -2.0320],
         [-0.2830, -0.9417, 62.7670,  ...,  0.5663, -0.9786, -0.9961]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 0.6833, -0.2425, -0.7981,  ...,  0.1691,  0.2525, -0.0896],
          [ 0.4186, -0.9559,  0.2865,  ...,  0.2985, -0.2212, -0.5420],
          [ 1.0518, -1.0907, -0.2447,  ..., -0.1912,  0.6064,  0.6801],
          ...,
          [ 0.3815, -0.1635, -0.3790,  ...,  0.7586, -0.1006,  0.9094],
          [ 0.4064,  0.9045,  0.2225,  ..., -0.2729, -0.0916, -0.6946],
          [ 0.4343, -0.4388, -0.5212

In [17]:
outputs = XGLM_model.generate(**inputs, max_length=50,
            num_beams=2,
            early_stopping=True)

decoded = [XGLM_tokenizer.decode(ids) for ids in outputs]

decoded


['</s> Emma and Monica are not doctors, but Mark and Andrew are. Who are doctors? Emma and Monica are doctors, but Mark and Andrew are doctors.</s>']

In [29]:
XGLM_pipe(context + ' ' + question + ' A: ', return_full_text=True, max_new_tokens=15)

[{'generated_text': 'Emma and Monica are not doctors, but Mark and Andrew are. Who are doctors? A:  Mark and Andrew are doctors.'}]

: 

In [ ]:
# These parameters will return suggestions, and only the newly created text making it easier for prompting suggestions.
mGPT_pipe("My tart needs some", num_return_sequences=4, return_full_text=False)